<p style="padding:4px;background-color:#eeeeee;
          border-radius:40px;margin:0;color:#111111;
          border:18px ridge #bbbbbb;
          font-family:Charter;font-weight: bold;font-size:280%;text-align:center;overflow:hidden;font-weight:500">💎 Dimonds Knapsack Problem 💰</p>

The knapsack problem is a classic optimization problem in computer science and operations research. It involves selecting a subset of items to pack into a knapsack with a limited weight capacity, while maximizing the total value of the selected items.

In this notebook, we will explore how to solve the knapsack problem using Linear Programming from pulp library in Python. We will start by loading cpecific columns from our data such as carats and price. Then we will preprocess the data by searching for any inconsistancies, converting carats to kilograms and checking the weight of all gemstones. In the end we will use pulp library to search for optimal selection of items for out knapsack.

# <p style="padding:8px;background-color:#eeeeee;border-radius:40px;margin:0;color:#111111;border:6px ridge #bbbbbb;font-family:Charter;font-weight: bold;font-size:100%;text-align:center;overflow:hidden;font-weight:500">Importing Main Libraries</p>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# <p style="padding:8px;background-color:#eeeeee;border-radius:40px;margin:0;color:#111111;border:6px ridge #bbbbbb;font-family:Charter;font-weight: bold;font-size:100%;text-align:center;overflow:hidden;font-weight:500">Loading Dataset</p>

In [2]:
# Getting path of dataset
dataset_path = "/kaggle/input/diamonds/diamonds.csv"

<div style="background-color:#cce5ff; color:#1c2e4a; padding:10px; border-radius:10px">
    <span style="font-weight:bold">💬 Info:</span> Only two columns - price and carat - are required to solve this problem without loading the entire dataset into memory.
</div>

In [3]:
# Loading price and carat from dataset
df = pd.read_csv(dataset_path, usecols=["price","carat"])

# Displaying first 5 rows
df.head()

,carat,price
0,0.23,326
1,0.21,326
2,0.23,327
3,0.29,334
4,0.31,335


<div style="background-color:#D4EDDA; color:#155724; padding:10px; border-radius:10px">
    <span style="font-weight:bold">✅ Success:</span> As we can see, we correctly loaded needed columns.
</div>

# <p style="padding:8px;background-color:#eeeeee;border-radius:40px;margin:0;color:#111111;border:6px ridge #bbbbbb;font-family:Charter;font-weight: bold;font-size:100%;text-align:center;overflow:hidden;font-weight:500">Preprocessing</p>

### Printing Shape of Data

In [4]:
# shape of dataframe
df.shape

(53940, 2)

<div style="background-color:#cce5ff; color:#1c2e4a; padding:10px; border-radius:10px">
    <span style="font-weight:bold">💬 Info:</span> Our data has 53940 rows and 2 columns.
</div>

### Checking Number of Missing Values

In [5]:
# number of missing values
df.isna().sum()

carat    0
price    0
dtype: int64

<div style="background-color:#cce5ff; color:#1c2e4a; padding:10px; border-radius:10px">
    <span style="font-weight:bold">💬 Info:</span> Data has no missing values.
</div>

<div style="background-color:#cce5ff; color:#1c2e4a; padding:10px; border-radius:10px">
    <span style="font-weight:bold">💬 Info:</span> Becouse it would be easier to work on standars units of measurment I am gonna convert carats to kilograms.
</div>

### Converting carats to kilograms

In [6]:
# Creating new column
df['kilogram'] = df['carat'].apply(lambda x: x * 0.0002)

# Dropping old column
df = df.drop("carat", 1)

# Printing results
df.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """


,price,kilogram
0,326,0.000046
1,326,0.000042
2,327,0.000046
3,334,0.000058
4,335,0.000062


### Calculating the weight of all items

In [7]:
# Sum of kilogram column
df.kilogram.sum()

8.608174000000002

<div style="background-color:#cce5ff; color:#1c2e4a; padding:10px; border-radius:10px">
    <span style="font-weight:bold">💬 Info:</span> Weight of all gemstones tourns out to be quite small 😅.
</div>

<div style="background-color:#D4EDDA; color:#155724; padding:10px; border-radius:10px">
    <span style="font-weight:bold">✅ Success:</span> We have verified that the data is clean, converted carats to kilograms, and calculated the total weight of all gemstones which is 8.6kg.
</div>

# <p style="padding:8px;background-color:#eeeeee;border-radius:40px;margin:0;color:#111111;border:6px ridge #bbbbbb;font-family:Charter;font-weight: bold;font-size:100%;text-align:center;overflow:hidden;font-weight:500">Solving Task</p>

<div style="background-color:#cce5ff; color:#1c2e4a; padding:10px; border-radius:10px">
    <span style="font-weight:bold">💬 Info:</span> We will start by defining how much weight our knapsack can carry.
</div>

In [8]:
# Defining the maximum weight of knapsack
KNAPSACK_CAPACITY_KG = 5

<div style="background-color:#cce5ff; color:#1c2e4a; padding:10px; border-radius:10px">
    <span style="font-weight:bold">💬 Info:</span> Now we import necessary modules and use them to solve our problem
</div>

In [9]:
# Importing the necessary modules
from pulp import LpProblem, LpVariable, lpSum, LpMaximize

# Defining the weights and values of the items
values = df.price.tolist()
weights = df.kilogram.tolist()

# Defining the number of items
n_items = len(values)

# Creating a PuLP maximization problem
problem = LpProblem("Knapsack Problem", LpMaximize)

# Defining the decision variables
x = [LpVariable("x{}".format(i), lowBound=0, upBound=1, cat="Integer") for i in range(n_items)]

# Defining the objective function
problem += lpSum([values[i] * x[i] for i in range(n_items)])

# Defining the constraint for the knapsack capacity
problem += lpSum([weights[i] * x[i] for i in range(n_items)]) <= KNAPSACK_CAPACITY_KG

# Solving the problem
problem.solve()

/opt/conda/lib/python3.7/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.7/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/c7b2c60e73c74290ad4eff88bfdf41b2-pulp.mps max timeMode elapsed branch printingOptions all solution /tmp/c7b2c60e73c74290ad4eff88bfdf41b2-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 6 COLUMNS
At line 215767 RHS
At line 215769 BOUNDS
At line 269710 ENDATA
Problem MODEL has 1 rows, 53940 columns and 53940 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1.58149e+08 - 0.36 seconds
Cgl0004I processed model has 1 rows, 33408 columns (33408 integer (26593 of which binary)) and 33408 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 1 integers unsatisfied sum - 0.356436
Cbc0038I Solution found of -1.58148e+08
Cbc0038I Cleaned solution of -1.58148e+08
Cbc0038I Before mini branch and bound, 

1

<div style="background-color:#cce5ff; color:#1c2e4a; padding:10px; border-radius:10px">
    <span style="font-weight:bold">💬 Info:</span> Now we will get the results.
</div>

In [10]:
# Printing the solution
selected_items = []
total_value = 0
total_weight = 0
for i in range(n_items):
    if x[i].value() == 1:
        selected_items.append(i)
        total_value += values[i]
        total_weight += weights[i]

print("Total value:", total_value)
print("Total weight:", total_weight)

Total value: 158149464
Total weight: 4.999999999999971


<div style="background-color:#cce5ff; color:#1c2e4a; padding:10px; border-radius:10px">
    <span style="font-weight:bold">💬 Info:</span><br>As we can see all items that have been chosen have weight of 4.99kg which is bellow our 5kg threshold.<br>Also all those items combined have price of 158,149,464 dolars.
</div>

In [11]:
print("Printing few selected items(index):", selected_items[:10])

Printing few selected items(index): [109, 127, 135, 139, 167, 168, 173, 180, 181, 207]


<div style="background-color:#D4EDDA; color:#155724; padding:10px; border-radius:10px">
    <span style="font-weight:bold">✅ Success:</span> Using Linear Programming we solved the knapsack problem by maximizing the price of all items not exceeding the weight limit of a knapsack. 
</div>


# <p style="padding:8px;background-color:#eeeeee;border-radius:40px;margin:0;color:#111111;border:6px ridge #bbbbbb;font-family:Charter;font-weight: bold;font-size:100%;text-align:center;overflow:hidden;font-weight:500">Conclusion</p>

In this notebook, we explored how to solve the classic optimization problem of the knapsack problem using Linear Programming from the pulp library in Python. We started by loading specific columns from our dataset, preprocessing the data by converting carats to kilograms and checking for any inconsistencies. We then defined the maximum weight of the knapsack and imported necessary modules to solve our problem. We defined the weights and values of the items, created a PuLP maximization problem, defined decision variables, and the objective function, and added a constraint for the knapsack capacity. Finally, we solved the problem and printed the solution, showing the selected items and their total value and weight.

Overall, this notebook provides a clear demonstration of how to use Linear Programming to solve the knapsack problem, which is a common optimization problem in computer science and operations research. I hope code is well-commented and it is easy to understand the steps involved in the solution.